
# **CAP 5516 - Medical Image Computing (Spring 2025)**
Programming Assignment #2 (40 points) 

**Due: 3/10/2025, 11:59 PM (EST)**
 
#### **Deep Learning-based Brain Tumor Segmentation Using MRI**
 
**1. Dataset**
 
A subset of the data used in the 2016 and 2017 Brain Tumour Image Segmentation (BraTS) challenges [1-3] will be 
used for this programming assignment. A detailed description about the dataset can be found in [4] 
(Task01_BrainTumour). The dataset can be downloaded here: 
https://drive.google.com/drive/folders/1HqEgzS8BV2c7xYNrZdEAnrHk7osJJ--2   (Choose 
Task01_BrainTumour.tar) 
 
Once you download the dataset, there are three folders and one json file (see the figure below). “dataset.json” 
provides an overview about the dataset (e.g., MRI modality, label information for segmentation, number of training 
and testing samples, etc.). “imagesTr” is the folder for training samples. “labelsTr” is the folder for segmentation 
masks (labels) of the training samples. We do not need the “imageTs” folder (testing samples) since no ground truth 
label is provided.  
 


**2. Task** 
 
1) Data visualization  
 
Use ITK-SNAP software to visualize a few MRI samples and their corresponding segmentation masks.  
 
2) Brain tumor segmentation using U-Net 
 
Train 2D U-Net (process MRI slice by slice) or 3D U-Net (volumetric processing) for brain tumor segmentation. Do 
5-fold cross validation on the training set (“imagesTr” folder) and report the segmentation results (Dice score and 
Hausdorff Dist.) 
 
 
 
**3. What to report** 
 
1) Present a visualization example (data and segmentation mask) using the ITK-SNAP software or the code. 
2) Implementation details of the network. 
3) Use a table to list the segmentation results (Dice score and Hausdorff Dist.) for each fold, as well as the average 
results of 5-fold.  
 
You can use this code to compute Hausdorf distance:  
from medpy.metric import binary 
https://loli.github.io/medpy/_modules/medpy/metric/binary.html#hd 
 
  
 
Evaluation:  
background (label 0), necrotic and non-enhancing tumor (label 1), peritumoral edema (label 2) and GD-enhancing 
tumor (label 4). The segmentation accuracy is measured by the Dice score and the Hausdorff distance (95%) metrics 
for enhancing tumor region (ET, label 4), regions of the tumor core (TC, labels 1 and 4), and the whole tumor region 
(WT, labels 1,2 and 4). 


4) Present a few examples of your segmentation results (an example is given below) for qualitative analysis.  

  
 
**4. What to submit** 
 
(1) A report for this assignment. The quality of the report is important.  
(2) Clean code and clear instructions (e.g., a readme file) to reproduce your results. If you choose to host the code on 
GitHub, please provide the GitHub link.  
 
 
Code Examples 
 
1. TransBTS for brain tumor segmentation: https://github.com/Wenxuan-1119/TransBTS 
2. Brain tumor 3D segmentation with MONAI: https://github.com/Project-MONAI/tutorials/blob/master/3d_segmentation/brats_segmentation_3d.ipynb 
3. https://github.com/Project-MONAI/tutorials 


 
 
**References** 
 
[1] Bakas, S., Reyes, M., Int., E. & Menze, B. Identifying the Best Machine Learning Algorithms for Brain Tumor 
Segmentation, Progression Assessment, and Overall Survival Prediction in the BRATS Challenge. arXiv preprint 
arXiv:1811.02629 (2018).  
[2] Bakas, S. et al. Advancing The Cancer Genome Atlas glioma MRI collections with expert segmentation labels 
and radiomic features. Scientific Data 4, 1–13 (2017).  
[3] Menze, B. H. et al. The Multimodal Brain Tumor Image Segmentation Benchmark (BRATS). IEEE Transactions 
on Medical Imaging 34, 1993– 2024 (2015). 
[4] Simpson, Amber L., et al. "A large annotated medical image dataset for the development and evaluation of 
segmentation algorithms." arXiv preprint arXiv:1902.09063 (2019). 
 
The segmentation 
mask of your method 
for a slice.  
The ground truth 
(GT) segmentation 
mask for a slice.  


## 1. Import Packages


In [1]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install opencv-python
%pip install mlxtend
%pip install libgcc

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
import random
import cv2
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
import torchvision.transforms as tt
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torch.utils.data import random_split, DataLoader

from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

import os

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement libgcc (from versions: none)
ERROR: No matching distribution found for libgcc
Note: you may need to restart the kernel to use updated packages.


# Data Visualization using the ITK-SNAP Software

In [2]:
# Using the ITK-SNAP Software, we got a couple of images. First the original and then the segmented images

from PIL import Image
import os, os.path

imgs = []
path = "ITK_Images"
valid_images = [".jpg",".gif",".png",".tga"]
for f in os.listdir(path):
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    imgs.append(Image.open(os.path.join(path,f)))


for img in imgs:
    print(img)
    img.show()



<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=465x475 at 0x7723BCF9F760>
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=465x475 at 0x7723BCF9FF70>
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=466x475 at 0x7722657CD540>
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=443x464 at 0x7722657CD5A0>
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=443x464 at 0x7722657CD600>
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=443x464 at 0x7722657CD660>


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/malneyugnfl/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/malneyugnfl/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/malneyugnfl/snap/code/common/.cache/gio-modules/libgiolibproxy.so


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/malneyugnfl/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/malneyugnfl/snap/code/common/.cache/gio-modules/libgiolibproxy.so
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/malneyugnfl/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
Failed to register: GDBus.Error:org.freedesktop.DBus.Error.NoReply: Message recipient disconnected from message bus without replying
Failed 

# Load Dataset

In [ ]:
# Convert .nii files as .pkl files and realize data normalization